Level Sets and Gradients Jupyter Notebook

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, clear_output
try:
    import contourpy as cpy
except Exception:
    cpy = None
from functools import lru_cache

#### **The cell below is used to see the 2D level sets for certain 3D Visualizations.**

**Initialization Phase** 

In [5]:
def f_original(x, y):
    return 0.5 * np.sin(x) * np.cos(y) + 0.15 * (x**2 - y**2)
# Monkey saddle: threefold saddle with a flat critical point at origin
# Scaled for a comparable z-range on [-3, 3]^2
def f_monkey_saddle(x, y):
    return 0.06 * (x**3 - 3.0 * x * y**2)
# Elliptic paraboloid: smooth convex bowl, simple gradients
def f_paraboloid(x, y):
    return 0.12 * (x**2 + y**2)
def f_sine_product_n1(x, y):
    return np.sin(1/2 * np.pi * x) * np.sin(1/2 * np.pi * y)

In [ ]:
surface_funcs = {
    "Original (sin/cos + saddle)": f_original,
    "Monkey saddle": f_monkey_saddle,
    "Paraboloid": f_paraboloid,
    "Sine product": f_sine_product_n1
}
x = np.linspace(-3.0, 3.0, 160)
y = np.linspace(-3.0, 3.0, 160)
X, Y = np.meshgrid(x, y)
_default_key = "Original (sin/cos + saddle)"
# Cache surface data per function key: Z, grads, stats, contour generator
_surface_cache: dict[str, dict] = {}
# Initialize from default
Z = surface_funcs[_default_key](X, Y)
zmin, zmax = float(Z.min()), float(Z.max())
_cg_main = None
def _build_or_get_cache(key: str) -> dict:
    entry = _surface_cache.get(key)
    if entry is None:
        f = surface_funcs[key]
        Z_local = f(X, Y)
        zmin_local, zmax_local = float(Z_local.min()), float(Z_local.max())
        cg = None
        if cpy is not None:
            try:
                cg = cpy.contour_generator(x=x, y=y, z=Z_local, name="serial")
            except Exception:
                cg = None
        entry = {
            "Z": Z_local,
            "zmin": zmin_local,
            "zmax": zmax_local,
            "cg": cg,
            # grads are computed lazily
            "dZ_dx": None,
            "dZ_dy": None,
        }
        _surface_cache[key] = entry
    return entry
def compute_level_set_polylines(level: float) -> list[np.ndarray]:
    # Use contourpy when available for faster polyline extraction
    try:
        entry = _build_or_get_cache(surface_dropdown.value)
        cg = entry.get("cg")
        if cpy is not None and cg is not None:
            lines = cg.lines(float(level))
            return [np.asarray(seg, dtype=float) for seg in lines if np.asarray(seg).shape[0] > 1]
    except Exception:
        pass
    # Fallback: Matplotlib contour path extraction
    fig, ax = plt.subplots()
    cs = ax.contour(x, y, Z, levels=[level])
    paths: list[np.ndarray] = []
    try:
        if hasattr(cs, "allsegs") and cs.allsegs and len(cs.allsegs[0]) > 0:
            for seg in cs.allsegs[0]:
                v = np.asarray(seg)
                if v.shape[0] > 1:
                    paths.append(v)
        elif hasattr(cs, "collections") and cs.collections:
            for p in cs.collections[0].get_paths():
                v = p.vertices
                if v.shape[0] > 1:
                    paths.append(v)
    finally:
        plt.close(fig)
    return paths
def get_current_f():
    return surface_funcs[surface_dropdown.value]
def partial_derivatives(x0: float, y0: float, h: float = 1e-3) -> tuple[float, float, float]:
    f = get_current_f()
    z0 = float(f(x0, y0))
    fx = float((f(x0 + h, y0) - f(x0 - h, y0)) / (2.0 * h))
    fy = float((f(x0, y0 + h) - f(x0, y0 - h)) / (2.0 * h))
    return z0, fx, fy
def _get_surface_grads(entry: dict) -> tuple[np.ndarray, np.ndarray]:
    if entry["dZ_dx"] is None or entry["dZ_dy"] is None:
        dZ_dy, dZ_dx = np.gradient(entry["Z"], y, x)
        entry["dZ_dx"], entry["dZ_dy"] = dZ_dx, dZ_dy
    return entry["dZ_dx"], entry["dZ_dy"]
def add_tangent_traces(fig: go.Figure, x0: float, y0: float, half_len: float = 0.8, npts: int = 60) -> None:
    z0, fx, fy = partial_derivatives(x0, y0)
    xs = np.linspace(max(float(x.min()), x0 - half_len), min(float(x.max()), x0 + half_len), npts)
    ys = np.linspace(max(float(y.min()), y0 - half_len), min(float(y.max()), y0 + half_len), npts)
    z_tan_x = z0 + fx * (xs - x0)
    z_tan_y = z0 + fy * (ys - y0)
    fig.add_trace(go.Scatter3d(x=[x0], y=[y0], z=[z0],mode="markers",marker=dict(size=5, color="#111111"),name="Point (x0, y0, f)"))
    fig.add_trace(go.Scatter3d(x=xs,y=np.full_like(xs, y0),z=z_tan_x,mode="lines",line=dict(color="#1f77b4", width=6, dash="dash"),name="dz/dx",showlegend=True,))
    fig.add_trace(go.Scatter3d(x=np.full_like(ys, x0),y=ys,z=z_tan_y,mode="lines",line=dict(color="#ff7f0e", width=6, dash="dash"),name="dz/dy",showlegend=True,))
def add_tangent_plane(fig: go.Figure, x0: float, y0: float, half_size: float = 0.8, resolution: int = 24, opacity: float = 0.4) -> None:
    z0, fx, fy = partial_derivatives(x0, y0)
    xp = np.linspace(max(float(x.min()), x0 - half_size), min(float(x.max()), x0 + half_size), resolution)
    yp = np.linspace(max(float(y.min()), y0 - half_size), min(float(y.max()), y0 + half_size), resolution)
    XP, YP = np.meshgrid(xp, yp)
    ZP = z0 + fx * (XP - x0) + fy * (YP - y0)
    fig.add_trace(go.Surface(x=XP,y=YP,z=ZP,colorscale=[[0, "#8a2be2"], [1, "#8a2be2"]],showscale=False,opacity=opacity,name="Tangent plane",showlegend=False,))
def add_normal_line(fig: go.Figure, x0: float, y0: float, length: float = 1.5) -> None:
    z0, fx, fy = partial_derivatives(x0, y0)
    v = np.array([fx, fy, -1.0])
    nrm = float(np.linalg.norm(v))
    if nrm == 0.0:
        nrm = 1.0
    v = v / nrm
    p1 = np.array([x0, y0, z0]) - 0.5 * length * v
    p2 = np.array([x0, y0, z0]) + 0.5 * length * v
    fig.add_trace(go.Scatter3d(x=[p1[0], p2[0]],y=[p1[1], p2[1]],z=[p1[2], p2[2]],mode="lines",line=dict(color="#2ca02c", width=6), name="Normal line",showlegend=True,))
def add_gradient_vector(fig: go.Figure, x0: float, y0: float, length: float = 2.0, color: str = "#e31a1c") -> None:
    z0, fx, fy = partial_derivatives(x0, y0)
    v = np.array([fx, fy, fx * fx + fy * fy], dtype=float)
    nrm = float(np.linalg.norm(v))
    if nrm < 1e-12:
        return
    dir_v = v / nrm
    p0 = np.array([x0, y0, z0])
    p1 = p0 + length * dir_v
    # Lifted visual arrow (not the planar gradient)
    fig.add_trace(go.Scatter3d(x=[p0[0], p1[0]],y=[p0[1], p1[1]],z=[p0[2], p1[2]],mode="lines",line=dict(color='#800080', width=12),name="Lifted ∇f direction",showlegend=True,))
    # Optional 3D cone at the tip
    if 'show_cones_chk' in globals() and getattr(show_cones_chk, 'value', False):
        try:
            fig.add_trace(go.Cone(x=[p1[0]],y=[p1[1]],z=[p1[2]],u=[dir_v[0]],v=[dir_v[1]],w=[dir_v[2]],anchor="tip",colorscale=[[0, '#800080'], [1, '#800080']],showscale=False,sizemode="absolute",sizeref=0.28,name="",))
        except Exception:
            pass
    # Planar projection of gradient onto the bottom floor (true ∇f in XY)
    mag_xy = float(np.hypot(fx, fy))
    if mag_xy < 1e-12:
        return
    dir_xy = np.array([fx, fy], dtype=float) / mag_xy
    z_floor = zmin + 1e-3
    p0_xy = np.array([x0, y0, z_floor], dtype=float)
    p1_xy = np.array([x0 + length * dir_xy[0], y0 + length * dir_xy[1], z_floor], dtype=float)
    fig.add_trace(go.Scatter3d(x=[p0_xy[0], p1_xy[0]],y=[p0_xy[1], p1_xy[1]],z=[p0_xy[2], p1_xy[2]],mode="lines",line=dict(color=color, width=10),name="Gradient ∇f",showlegend=True,))
    # Optional planar cone
    if 'show_cones_chk' in globals() and getattr(show_cones_chk, 'value', False):
        try:
            fig.add_trace(go.Cone(x=[p1_xy[0]],y=[p1_xy[1]],z=[p1_xy[2]],u=[dir_xy[0]],v=[dir_xy[1]],w=[0.0],anchor="tip",colorscale=[[0, color], [1, color]],showscale=False,sizemode="absolute",sizeref=0.24,name="",))
        except Exception:
            pass
def add_projection_connector(fig: go.Figure, x0: float, y0: float, color: str = "rgba(0,0,0,0.5)", width: int = 3, dash: str = "longdashdot") -> None:
    z0, _, _ = partial_derivatives(x0, y0)
    z_floor = zmin + 1e-3
    fig.add_trace(go.Scatter3d(x=[x0, x0],y=[y0, y0],z=[z_floor, z0],mode="lines",line=dict(color=color, width=width, dash=dash),name="",showlegend=False,))
def add_gradient_field_flat(fig: go.Figure, density: int = 12, arrow_color: str = "#1f77b4", arrow_length: float = 0.6, head_length_frac: float = 0.25, head_angle_deg: float = 28.0, line_width: int = 6) -> None:
    # Ensure Z reflects the current surface and get cached grads
    _update_z_stats_for_current_surface()
    entry = _build_or_get_cache(surface_dropdown.value)
    dZ_dx, dZ_dy = _get_surface_grads(entry)
    ny, nx = entry["Z"].shape
    step_x = max(1, nx // density)
    step_y = max(1, ny // density)
    xs = X[::step_y, ::step_x]
    ys = Y[::step_y, ::step_x]
    fx_sampled = dZ_dx[::step_y, ::step_x]
    fy_sampled = dZ_dy[::step_y, ::step_x]
    mags = np.sqrt(fx_sampled * fx_sampled + fy_sampled * fy_sampled) + 1e-9
    ux = fx_sampled / mags
    uy = fy_sampled / mags
    # Prepare multi-segment lines with NaN breaks
    z_floor = float(zmin + 1e-3)
    x_lines = []
    y_lines = []
    z_lines = []
    x_heads = []
    y_heads = []
    z_heads = []
    head_len = float(arrow_length * head_length_frac)
    theta = float(np.deg2rad(head_angle_deg))
    cos_t, sin_t = float(np.cos(theta)), float(np.sin(theta))
    def rot(u, v, c, s):
        return u * c - v * s, u * s + v * c
    for j in range(xs.shape[0]):
        for i in range(xs.shape[1]):
            x0 = float(xs[j, i])
            y0 = float(ys[j, i])
            dx = float(ux[j, i])
            dy = float(uy[j, i])
            x1 = x0 + arrow_length * dx
            y1 = y0 + arrow_length * dy
            x_lines.extend([x0, x1, np.nan])
            y_lines.extend([y0, y1, np.nan])
            z_lines.extend([z_floor, z_floor, np.nan])
            rx1, ry1 = rot(dx, dy, cos_t, sin_t)
            rx2, ry2 = rot(dx, dy, cos_t, -sin_t)
            x_heads.extend([x1, x1 - head_len * rx1, np.nan])
            y_heads.extend([y1, y1 - head_len * ry1, np.nan])
            z_heads.extend([z_floor, z_floor, np.nan])
            x_heads.extend([x1, x1 - head_len * rx2, np.nan])
            y_heads.extend([y1, y1 - head_len * ry2, np.nan])
            z_heads.extend([z_floor, z_floor, np.nan])
    fig.add_trace(go.Scatter3d(x=x_lines,y=y_lines,z=z_lines,mode="lines",line=dict(color=arrow_color, width=line_width),name="Gradient field",showlegend=True,))
    fig.add_trace(go.Scatter3d(x=x_heads,y=y_heads,z=z_heads,mode="lines",line=dict(color=arrow_color, width=line_width),name="",showlegend=False,))
def build_3d_figure(level_z: float, show_plane: bool, plane_z: float, birds_eye: bool, bottom_mode: str) -> go.Figure:
    fig = go.Figure()
    # Main surface
    fig.add_trace(go.Surface(x=X,y=Y,z=Z,colorscale="Viridis",reversescale=False,showscale=False,colorbar=dict(title="Height"),name="Surface",opacity=0.55,))
    # Optional horizontal plane at z = plane_z
    if show_plane:
        plane_z_arr = np.full_like(Z, plane_z)
        fig.add_trace(go.Surface(x=X,y=Y,z=plane_z_arr,colorscale=[[0, "#AAAAAA"], [1, "#AAAAAA"]],showscale=False,opacity=0.30,name=f"Plane z={plane_z:.2f}",))
    # Highlight the intersection contour at the selected level
    level_paths = compute_level_set_polylines(level_z)
    for verts in level_paths:
        fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], level_z),mode="lines",line=dict(color="#FF4136", width=6),name=f"Contour at z={level_z:.2f}",showlegend=False,))
    # Bottom content selection driven by new checkboxes
    z_floor = zmin
    if show_bottom_heatmap_chk.value:
        fig.add_trace(go.Surface(x=X,y=Y,z=np.full_like(Z, z_floor),surfacecolor=Z,cmin=zmin,cmax=zmax,colorscale="Viridis",showscale=False,opacity=0.4,name="Topo floor",hoverinfo="skip",))
        if zmax == zmin:
            selected_levels = [zmin]
        else:
            z_span = (zmax - zmin)
            selected_levels = list(zmin + np.linspace(0.05, 0.95, 10) * z_span)
        for lvl in selected_levels:
            for verts in compute_level_set_polylines(lvl):
                fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], z_floor + 1e-3), mode="lines",line=dict(color="#555555", width=5),name="Topo contours",showlegend=False,))
    if show_bottom_arrows_chk.value:
        add_gradient_field_flat(fig, density=12, arrow_color="#1f77b4", arrow_length=0.2, head_length_frac=0.28, head_angle_deg=26.0, line_width=6)
    if show_bottom_redlevel_chk.value:
        for verts in level_paths:
            fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], z_floor + 1e-3),mode="lines",line=dict(color="#FF4136", width=5),name="Selected level (floor)",showlegend=False,))
    scene = dict(xaxis_title="x",yaxis_title="y",zaxis_title="z",xaxis=dict(showspikes=False),yaxis=dict(showspikes=False),zaxis=dict(showspikes=False),aspectmode="data",
    )
    if birds_eye:
        fig.update_layout(scene=dict(**scene, camera=dict(eye=dict(x=0.0001, y=0.0001, z=2.5), projection=dict(type="orthographic"))),margin=dict(l=0, r=0, t=0, b=100),legend=dict(orientation="h", y=-0.12, yanchor="top", x=0.5, xanchor="center"),title=f"3D View (Bird's-eye camera) — level: {level_z:.2f}",width=1100,height=800,uirevision="main-3d")
    else:
        fig.update_layout(scene=dict(**scene, camera=dict(eye=dict(x=1.35, y=1.35, z=0.95), projection=dict(type="orthographic"))), margin=dict(l=0, r=0, t=100, b=100),legend=dict(orientation="h", y=-0.12, yanchor="top", x=0.5, xanchor="center"),title=f"3D Visual Representation of the Gradient",width=1100,height=800,uirevision="main-3d")
    return fig
instructions = widgets.HTML(
    value=(
        "<b>How to use:</b>"
        "<ul>"
        "<li>Select a surface from the dropdown to switch functions.</li>"
        "<li>Rotate/zoom the 3D surface. It's colored by height, with the intersection contour highlighted in red.</li>"
        "<li>Optionally show a floor topo heatmap + contours to view from above.</li>"
        "<li>Enter a point (x0, y0) and press Enter to add tangent lines, the tangent plane, and the normal line at that point.</li>"
        "</ul>"
    )
)
surface_dropdown = widgets.Dropdown(options=list(surface_funcs.keys()),value=_default_key,description="Surface",layout=widgets.Layout(width="280px"),
)
z_slider = widgets.FloatSlider(
    description="Level/Plane z",min=zmin,max=zmax,step=(zmax - zmin) / 200.0 if zmax > zmin else 0.01,value=(zmin + zmax) / 2.0,continuous_update=False,readout_format=".2f",layout=widgets.Layout(width="350px"),
)
show_plane_chk = widgets.Checkbox(value=False, description="Show plane")
birds_eye_toggle = widgets.ToggleButton(
    value=False, description="Bird’s-eye 2D view", icon="eye"
)
bottom_mode_dd = widgets.Dropdown(options=["No Bottom Floor", "Level set heatmap", "Gradient vector field", "Heatmap + gradient field"],value="No Bottom Floor",description="Bottom",layout=widgets.Layout(width="350px"),
)
# Hide legacy dropdown in favor of mix-and-match controls
bottom_mode_dd.layout.display = "none"
# New: mix-and-match bottom plane controls ("Alter Bottom Plane")
show_bottom_heatmap_chk = widgets.Checkbox(value=False, description="Heatmap")
show_bottom_arrows_chk = widgets.Checkbox(value=False, description="Gradient field")
show_bottom_redlevel_chk = widgets.Checkbox(value=False, description="Selected level (red)")
bottom_table_title = widgets.HTML("<b>Alter Bottom Plane</b>")
bottom_table = widgets.VBox([widgets.HBox([show_bottom_heatmap_chk, widgets.HTML("Level sets heatmap")], layout=widgets.Layout(align_items="center")),widgets.HBox([show_bottom_arrows_chk, widgets.HTML("Gradient vector field")], layout=widgets.Layout(align_items="center")),widgets.HBox([show_bottom_redlevel_chk, widgets.HTML("Red level set projection")], layout=widgets.Layout(align_items="center")),
], layout=widgets.Layout(align_items="flex-start"))
# New: lock the level to f(x0,y0)
lock_level_chk = widgets.Checkbox(value=True, description="Lock level set to f(x0,y0)")
x0_input = widgets.FloatText(description="x0", value=0.5, step=0.05, layout=widgets.Layout(width="180px"))
y0_input = widgets.FloatText(description="y0", value=0.5, step=0.05, layout=widgets.Layout(width="180px"))
show_tangent_plane_chk = widgets.Checkbox(value=False, description="Show tangent plane")
# New: toggle for cone arrowheads (off by default for performance)
show_cones_chk = widgets.Checkbox(value=False, description="Show arrowheads (cones)")
out3d = widgets.Output()
out3d.layout = widgets.Layout(width="1150px", height="820px")
out2d = widgets.Output()
current_fig3d = None
current_fig2d = None
def _update_z_stats_for_current_surface():
    global Z, zmin, zmax, _cg_main
    # Get or build cache for current surface
    entry = _build_or_get_cache(surface_dropdown.value)
    Z = entry["Z"]
    zmin, zmax = entry["zmin"], entry["zmax"]
    _cg_main = entry.get("cg")
    # update unified slider
    z_slider.min = zmin
    z_slider.max = zmax
    z_slider.step = (zmax - zmin) / 200.0 if zmax > zmin else 0.01
    if z_slider.value < zmin or z_slider.value > zmax:
        z_slider.value = (zmin + zmax) / 2.0
is_rendering_main = False
def render_all():
    global current_fig3d, current_fig2d, is_rendering_main
    if is_rendering_main:
        return
    is_rendering_main = True
    _update_z_stats_for_current_surface()
    # Determine the level to use
    try:
        x0v = float(x0_input.value)
        y0v = float(y0_input.value)
    except Exception:
        x0v, y0v = 0.0, 0.0
    if lock_level_chk.value:
        try:
            f = get_current_f()
            level_val = float(np.clip(f(x0v, y0v), zmin, zmax))
        except Exception:
            level_val = float(np.clip((zmin + zmax) / 2.0, zmin, zmax))
        z_slider.layout.display = "none"
    else:
        level_val = z_slider.value
        z_slider.layout.display = "flex"
    current_fig3d = build_3d_figure(
        level_z=level_val,
        show_plane=show_plane_chk.value,
        plane_z=level_val,
        birds_eye=birds_eye_toggle.value,
        bottom_mode=bottom_mode_dd.value,
    )
    try:
        if np.isfinite(x0v) and np.isfinite(y0v):
            if show_tangent_plane_chk.value:
                add_tangent_plane(current_fig3d, x0v, y0v, half_size=0.9, resolution=28, opacity=0.35)
            add_normal_line(current_fig3d, x0v, y0v, length=1.6)
            add_tangent_traces(current_fig3d, x0v, y0v, half_len=0.9)
            add_projection_connector(current_fig3d, x0v, y0v)
            # Prominent gradient vector at (x0, y0)
            add_gradient_vector(current_fig3d, x0v, y0v, length=.75, color="#e31a1c")
    except Exception:
        pass
    with out3d:
        clear_output(wait=True)
        display(current_fig3d)
    is_rendering_main = False
surface_dropdown.observe(lambda change: render_all(), names="value")
z_slider.observe(lambda change: render_all(), names="value")
show_plane_chk.observe(lambda change: render_all(), names="value")
birds_eye_toggle.observe(lambda change: render_all(), names="value")
bottom_mode_dd.observe(lambda change: render_all(), names="value")
# Observe new bottom toggles
show_bottom_heatmap_chk.observe(lambda change: render_all(), names="value")
show_bottom_arrows_chk.observe(lambda change: render_all(), names="value")
show_bottom_redlevel_chk.observe(lambda change: render_all(), names="value")
x0_input.observe(lambda change: render_all(), names="value")
y0_input.observe(lambda change: render_all(), names="value")
show_tangent_plane_chk.observe(lambda change: render_all(), names="value")
lock_level_chk.observe(lambda change: render_all(), names="value")
show_cones_chk.observe(lambda change: render_all(), names="value")
controls_row1 = widgets.HBox([
    surface_dropdown,
])
plane_controls = widgets.HBox([
    show_plane_chk, z_slider, bottom_mode_dd, lock_level_chk,
])
# Replace dropdown with the new "Alter Bottom Plane" table in the UI
plane_controls_mx = widgets.VBox([
    widgets.HBox([show_plane_chk, z_slider, lock_level_chk]),
    bottom_table_title,
    bottom_table,
])
point_row = widgets.HBox([widgets.HTML("<b>Point (press Enter):</b>&nbsp;"),x0_input,y0_input,show_tangent_plane_chk, show_cones_chk])
ui = widgets.VBox([instructions,controls_row1,plane_controls_mx,widgets.HBox([birds_eye_toggle]),point_row,out3d,out2d,
])


In [7]:
# 3D Gradient Field view (copied from `ui` but with arrows on the bottom instead of plane/floor/contour)
out3d_grad = widgets.Output()
out3d_grad.layout = widgets.Layout(width="1150px", height="820px")
# Local controls for this view
surface_dropdown_g = widgets.Dropdown(options=list(surface_funcs.keys()),value=_default_key,description="Surface",layout=widgets.Layout(width="280px"),)
x0_input_g = widgets.FloatText(description="x0", value=0.5, step=0.05, layout=widgets.Layout(width="180px"))
y0_input_g = widgets.FloatText(description="y0", value=0.5, step=0.05, layout=widgets.Layout(width="180px"))
show_tangent_plane_chk_g = widgets.Checkbox(value=False, description="Show tangent plane")
birds_eye_toggle_g = widgets.ToggleButton(value=False, description="Bird’s-eye 2D view", icon="eye")
def _sync_main_from_grad_controls():
    # Keep main controls in sync so shared functions use correct state
    try:
        if surface_dropdown.value != surface_dropdown_g.value:
            surface_dropdown.value = surface_dropdown_g.value
        if float(x0_input.value) != float(x0_input_g.value):
            x0_input.value = float(x0_input_g.value)
        if float(y0_input.value) != float(y0_input_g.value):
            y0_input.value = float(y0_input_g.value)
        if show_tangent_plane_chk.value != show_tangent_plane_chk_g.value:
            show_tangent_plane_chk.value = show_tangent_plane_chk_g.value
        if birds_eye_toggle.value != birds_eye_toggle_g.value:
            birds_eye_toggle.value = birds_eye_toggle_g.value
    except Exception:
        pass
def _sync_grad_controls_from_main(*args, **kwargs):
    try:
        surface_dropdown_g.value = surface_dropdown.value
        x0_input_g.value = float(x0_input.value)
        y0_input_g.value = float(y0_input.value)
        show_tangent_plane_chk_g.value = show_tangent_plane_chk.value
        birds_eye_toggle_g.value = birds_eye_toggle.value
    except Exception:
        pass
def add_gradient_field(fig: go.Figure, density: int = 12, arrow_color: str = "#1f77b4") -> None:
    # Use the flat floor arrows helper for consistency
    add_gradient_field_flat(fig, density=density, arrow_color=arrow_color, arrow_length=0.2, head_length_frac=0.28, head_angle_deg=26.0, line_width=6)
def build_3d_figure_grad(birds_eye: bool) -> go.Figure:
    fig = go.Figure()
    # Main surface only
    fig.add_trace(go.Surface(x=X,y=Y,z=Z,colorscale="Viridis",reversescale=False,showscale=False,colorbar=dict(title="Height"),name="Surface",opacity=0.55,))
    # Replace bottom plane with gradient arrows
    add_gradient_field(fig, density=14, arrow_color="#1f77b4")
    scene = dict(xaxis_title="x",yaxis_title="y",zaxis_title="z",xaxis=dict(showspikes=False),yaxis=dict(showspikes=False),zaxis=dict(showspikes=False),aspectmode="data",)
    if birds_eye:
        fig.update_layout(scene=dict(**scene, camera=dict(eye=dict(x=0.0001, y=0.0001, z=2.5), projection=dict(type="orthographic"))),margin=dict(l=0, r=0, t=0, b=100),legend=dict(orientation="h", y=-0.12, yanchor="top", x=0.5, xanchor="center"),title=f"3D Gradient Field",width=1100,height=800)
    else:
        fig.update_layout(scene=dict(**scene, camera=dict(eye=dict(x=1.35, y=1.35, z=0.95), projection=dict(type="orthographic"))), margin=dict(l=0, r=0, t=100, b=100),legend=dict(orientation="h", y=-0.12, yanchor="top", x=0.5, xanchor="center"),title=f"3D Gradient Field",width=1100,height=800)
    return fig
def render_grad_view():
    # Sync main controls from local gradient controls first
    _sync_main_from_grad_controls()
    # Sync surface stats
    _update_z_stats_for_current_surface()
    fig = build_3d_figure_grad(birds_eye=birds_eye_toggle.value)
    try:
        x0v = float(x0_input.value)
        y0v = float(y0_input.value)
        if np.isfinite(x0v) and np.isfinite(y0v):
            if show_tangent_plane_chk.value:
                add_tangent_plane(fig, x0v, y0v, half_size=0.9, resolution=28, opacity=0.35)
            add_normal_line(fig, x0v, y0v, length=1.6)
            add_tangent_traces(fig, x0v, y0v, half_len=0.9)
            add_projection_connector(fig, x0v, y0v)
            # Draw selected level set at z = f(x0,y0) on surface and floor
            try:
                f = get_current_f()
                level_val = float(np.clip(f(x0v, y0v), zmin, zmax))
            except Exception:
                level_val = float(np.clip((zmin + zmax) / 2.0, zmin, zmax))
            level_paths = compute_level_set_polylines(level_val)
            for verts in level_paths:
                fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], level_val),mode="lines",line=dict(color="#FF4136", width=6),name=f"Contour at z={level_val:.2f}",showlegend=False,))
            z_floor = zmin
            for verts in level_paths:
                fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], z_floor + 1e-3),mode="lines",line=dict(color="#FF4136", width=2.5),name="Selected level (floor)",showlegend=False,))
            add_gradient_vector(fig, x0v, y0v, length=.6, color="#e31a1c")
    except Exception:
        pass
    with out3d_grad:
        clear_output(wait=True)
        display(fig)
# Re-render when relevant controls change (both local and main)
surface_dropdown_g.observe(lambda change: render_grad_view(), names="value")
x0_input_g.observe(lambda change: render_grad_view(), names="value")
y0_input_g.observe(lambda change: render_grad_view(), names="value")
show_tangent_plane_chk_g.observe(lambda change: render_grad_view(), names="value")
birds_eye_toggle_g.observe(lambda change: render_grad_view(), names="value")
surface_dropdown.observe(lambda change: (_sync_grad_controls_from_main(), render_grad_view()), names="value")
birds_eye_toggle.observe(lambda change: (_sync_grad_controls_from_main(), render_grad_view()), names="value")
x0_input.observe(lambda change: (_sync_grad_controls_from_main(), render_grad_view()), names="value")
y0_input.observe(lambda change: (_sync_grad_controls_from_main(), render_grad_view()), names="value")
show_tangent_plane_chk.observe(lambda change: (_sync_grad_controls_from_main(), render_grad_view()), names="value")

In [8]:
# Clean 3D Level Sets UI (no tangent plane, gradient, derivative lines, normal, or point)
# Controls mirroring the main 3D UI
surface_dropdown_ls = widgets.Dropdown(options=list(surface_funcs.keys()),value=_default_key,description="Surface",layout=widgets.Layout(width="280px"),)
z_slider_ls = widgets.FloatSlider(description="Level/Plane z",min=-1.0,max=1.0,step=0.01,value=0.0,continuous_update=False,readout_format=".2f",layout=widgets.Layout(width="350px"),)
show_plane_chk_ls = widgets.Checkbox(value=False, description="Show plane")
birds_eye_toggle_ls = widgets.ToggleButton(value=False, description="Bird’s-eye 2D view", icon="eye")
show_heatmap_chk_ls = widgets.Checkbox(value=True, description="Show topo floor")
out3d_ls = widgets.Output()
out3d_ls.layout = widgets.Layout(width="1150px", height="820px")
# Local state for the clean view
Z_ls = surface_funcs[_default_key](X, Y)
zmin_ls, zmax_ls = float(Z_ls.min()), float(Z_ls.max())
# Fast contour generator cache for clean view
_cg_ls = None
def _update_z_stats_for_current_surface_ls():
    global Z_ls, zmin_ls, zmax_ls, _cg_ls
    f = surface_funcs[surface_dropdown_ls.value]
    Z_ls = f(X, Y)
    zmin_ls, zmax_ls = float(Z_ls.min()), float(Z_ls.max())
    z_slider_ls.min = zmin_ls
    z_slider_ls.max = zmax_ls
    z_slider_ls.step = (zmax_ls - zmin_ls) / 200.0 if zmax_ls > zmin_ls else 0.01
    if z_slider_ls.value < zmin_ls or z_slider_ls.value > zmax_ls:
        z_slider_ls.value = (zmin_ls + zmax_ls) / 2.0
    # Build a contourpy generator for fast level sets if available
    _cg_ls = None
    if cpy is not None:
        try:
            _cg_ls = cpy.contour_generator(x=x, y=y, z=Z_ls, name="serial")
        except Exception:
            _cg_ls = None
def compute_level_set_polylines_ls(level: float, Z_arr: np.ndarray) -> list[np.ndarray]:
    # Prefer contourpy if available and generator built
    if cpy is not None and _cg_ls is not None:
        try:
            lines = _cg_ls.lines(float(level))
            return [np.asarray(seg, dtype=float) for seg in lines if np.asarray(seg).shape[0] > 1]
        except Exception:
            pass
    # Fallback to Matplotlib if contourpy path fails
    fig, ax = plt.subplots()
    cs = ax.contour(x, y, Z_arr, levels=[level])
    paths = []
    try:
        if hasattr(cs, "allsegs") and cs.allsegs and len(cs.allsegs[0]) > 0:
            for seg in cs.allsegs[0]:
                v = np.asarray(seg)
                if v.shape[0] > 1:
                    paths.append(v)
        elif hasattr(cs, "collections") and cs.collections:
            for p in cs.collections[0].get_paths():
                v = p.vertices
                if v.shape[0] > 1:
                    paths.append(v)
    finally:
        plt.close(fig)
    return paths
def build_3d_figure_ls(level_z: float, show_plane: bool, plane_z: float, birds_eye: bool, show_floor: bool) -> go.Figure:
    fig = go.Figure()
    # Main surface
    fig.add_trace(go.Surface(x=X,y=Y,z=Z_ls,colorscale="Viridis",reversescale=False,showscale=False,colorbar=dict(title="Height"),name="Surface",opacity=0.55,))
    # Optional horizontal plane at z = plane_z
    if show_plane:
        plane_z_arr = np.full_like(Z_ls, plane_z)
        fig.add_trace(go.Surface(x=X,y=Y,z=plane_z_arr,colorscale=[[0, "#AAAAAA"], [1, "#AAAAAA"]],showscale=False,opacity=0.30,name=f"Plane z={plane_z:.2f}",))
    # Highlight the intersection contour at the selected level
    level_paths = compute_level_set_polylines_ls(level_z, Z_ls)
    for verts in level_paths:
        fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], level_z),mode="lines",line=dict(color="#FF4136", width=6),name=f"Contour at z={level_z:.2f}",showlegend=False,))
    # Optional topo floor at z = zmin with heatmap + contour lines
    if show_floor:
        z_floor = zmin_ls
        fig.add_trace(go.Surface(x=X,y=Y,z=np.full_like(Z_ls, z_floor),surfacecolor=Z_ls,cmin=zmin_ls,cmax=zmax_ls,colorscale="Viridis",showscale=False,opacity=0.4,name="Topo floor",hoverinfo="skip",))
        if zmax_ls == zmin_ls:
            selected_levels = [zmin_ls]
        else:
            z_span = (zmax_ls - zmin_ls)
            selected_levels = list(zmin_ls + np.linspace(0.05, 0.95, 10) * z_span)
        for lvl in selected_levels:
            for verts in compute_level_set_polylines_ls(lvl, Z_ls):
                fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], z_floor + 1e-3),mode="lines",line=dict(color="#555555", width=2.5),name="Topo contours",showlegend=False,))
        # Project the selected level set onto the floor in red (thin)
        for verts in level_paths:
            fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], z_floor + 1e-3),mode="lines",line=dict(color="#FF4136", width=5),name="Selected level (floor)",showlegend=False,))
    scene = dict(xaxis_title="x",yaxis_title="y",zaxis_title="z",xaxis=dict(showspikes=False),yaxis=dict(showspikes=False),zaxis=dict(showspikes=False),aspectmode="data",)
    if birds_eye:
        fig.update_layout(scene=dict(**scene, camera=dict(eye=dict(x=0.0001, y=0.0001, z=2.5), projection=dict(type="orthographic"))),margin=dict(l=0, r=0, t=0, b=100),legend=dict(orientation="h", y=-0.12, yanchor="top", x=0.5, xanchor="center"),title=f"3D Level Sets (Clean) — level: {level_z:.2f}",width=1100,height=800,uirevision="levels-3d")
    else:
        fig.update_layout(scene=dict(**scene, camera=dict(eye=dict(x=1.35, y=1.35, z=0.95), projection=dict(type="orthographic"))),margin=dict(l=0, r=0, t=100, b=100),legend=dict(orientation="h", y=-0.12, yanchor="top", x=0.5, xanchor="center"),title=f"3D Level Sets (Clean)",width=1100,height=800,uirevision="levels-3d")
    return fig
is_rendering_ls = False
def render_levels():
    global is_rendering_ls
    if is_rendering_ls:
        return
    is_rendering_ls = True
    _update_z_stats_for_current_surface_ls()
    fig = build_3d_figure_ls(level_z=z_slider_ls.value,show_plane=show_plane_chk_ls.value,plane_z=z_slider_ls.value,birds_eye=birds_eye_toggle_ls.value,show_floor=show_heatmap_chk_ls.value,)
    with out3d_ls:
        clear_output(wait=True)
        display(fig)
    is_rendering_ls = False
# Wire observers (local controls)
surface_dropdown_ls.observe(lambda change: render_levels(), names="value")
z_slider_ls.observe(lambda change: render_levels(), names="value")
show_plane_chk_ls.observe(lambda change: render_levels(), names="value")
birds_eye_toggle_ls.observe(lambda change: render_levels(), names="value")
show_heatmap_chk_ls.observe(lambda change: render_levels(), names="value")
# Also sync with the main UI controls if they exist
def _sync_ls_from_main(*args, **kwargs):
    try:
        surface_dropdown_ls.value = surface_dropdown.value
        _update_z_stats_for_current_surface_ls()
        z_slider_ls.value = float(np.clip(z_slider.value, zmin_ls, zmax_ls))
        show_plane_chk_ls.value = show_plane_chk.value
        birds_eye_toggle_ls.value = birds_eye_toggle.value
        show_heatmap_chk_ls.value = show_heatmap_chk.value
    except Exception:
        pass
    render_levels()
try:
    surface_dropdown.observe(lambda change: _sync_ls_from_main(), names="value")
    z_slider.observe(lambda change: _sync_ls_from_main(), names="value")
    show_plane_chk.observe(lambda change: _sync_ls_from_main(), names="value")
    birds_eye_toggle.observe(lambda change: _sync_ls_from_main(), names="value")
    show_heatmap_chk.observe(lambda change: _sync_ls_from_main(), names="value")
except Exception:
    pass
controls_row1_ls = widgets.HBox([surface_dropdown_ls])
plane_controls_ls = widgets.HBox([show_plane_chk_ls, z_slider_ls, show_heatmap_chk_ls])

In [ ]:
# Interactive surface cross-section and linearization (Visualization 1)
# This cell builds a 3D surface with a movable point, a selectable cross-section
# (holding x or y constant), an optional linearization along that selected variable,
# and a 2D side panel showing the slice with the tangent line.
# Reuse existing state where possible
try:
    surface_dropdown_v1 = widgets.Dropdown(options=list(surface_funcs.keys()), value=surface_dropdown.value, description="Surface", layout=widgets.Layout(width="280px"))
except Exception:
    surface_dropdown_v1 = widgets.Dropdown(options=list(surface_funcs.keys()), value=list(surface_funcs.keys())[0], description="Surface", layout=widgets.Layout(width="280px"))
x0_v1 = widgets.FloatSlider(description="x0", min=float(x.min()), max=float(x.max()), step=0.02, value=-2.5, readout_format=".2f", continuous_update=False, layout=widgets.Layout(width="300px"))
y0_v1 = widgets.FloatSlider(description="y0", min=float(y.min()), max=float(y.max()), step=0.02, value=1.5, readout_format=".2f", continuous_update=False, layout=widgets.Layout(width="300px"))
show_linear_v1 = widgets.Checkbox(value=True, description="Show linearizations (x and y)")
COLOR_X = "#1f77b4"  # x-linearization color
COLOR_Y = "#ff7f0e"  # y-linearization color
out3d_v1 = widgets.Output()
out2d_v1 = widgets.Output()
out2d_x_v1 = widgets.Output()
out2d_y_v1 = widgets.Output()
# Helpers using the shared utilities from above
def _get_f_v1():
    try:
        return surface_funcs[surface_dropdown_v1.value]
    except Exception:
        return list(surface_funcs.values())[0]
def _slice_values_and_tangent(x0: float, y0: float, axis: str, span: float = 3.0, n: int = 220):
    f = _get_f_v1()
    z0, fx, fy = partial_derivatives(x0, y0)
    if axis == "x":
        # Hold x fixed, vary y
        ys = np.linspace(max(float(y.min()), y0 - span), min(float(y.max()), y0 + span), n)
        xs = np.full_like(ys, x0)
        zs = f(xs, ys)
        z_lin = z0 + fy * (ys - y0)  # linearization along y
        tvar = ys
        t0 = y0
        label = "Slice: x = const, vary y"
    else:
        # Hold y fixed, vary x
        xs = np.linspace(max(float(x.min()), x0 - span), min(float(x.max()), x0 + span), n)
        ys = np.full_like(xs, y0)
        zs = f(xs, ys)
        z_lin = z0 + fx * (xs - x0)  # linearization along x
        tvar = xs
        t0 = x0
        label = "Slice: y = const, vary x"
    return xs, ys, zs, z_lin, tvar, t0, z0, label
def _build_3d_v1():
    # Ensure Z, zmin, zmax reflect current surface
    _update_z_stats_for_current_surface()
    fig = go.Figure()
    # Surface
    fig.add_trace(go.Surface(x=X, y=Y, z=Z, colorscale="Viridis", showscale=False, opacity=float(0.3), name="Surface"))
    # Slices and linearizations for both coordinate directions
    x0 = float(x0_v1.value)
    y0 = float(y0_v1.value)
    # vary x (hold y): along x
    xs_y, ys_y, zs_y, z_lin_x, tvar_x, t0_x, z0, _ = _slice_values_and_tangent(x0, y0, "y")
    # vary y (hold x): along y
    xs_x, ys_x, zs_x, z_lin_y, tvar_y, t0_y, _, _ = _slice_values_and_tangent(x0, y0, "x")
    # Surface slices
    fig.add_trace(go.Scatter3d(x=xs_y, y=ys_y, z=zs_y, mode="lines", line=dict(color="#1f77b4", width=6), name="x-slice on surface"))
    fig.add_trace(go.Scatter3d(x=xs_x, y=ys_x, z=zs_x, mode="lines", line=dict(color="#ff7f0e", width=6), name="y-slice on surface"))
    # Movable point
    fig.add_trace(go.Scatter3d(x=[x0], y=[y0], z=[z0], mode="markers", marker=dict(size=6, color="#111111"), name="Point (x0, y0, f)"))
    # Linearization lines (dashed) for both directions
    if show_linear_v1.value:
        # x-linearization: vary x, hold y (line)
        fig.add_trace(go.Scatter3d(x=xs_y, y=np.full_like(xs_y, y0), z=z_lin_x, mode="lines", line=dict(color="#1f77b4", width=4, dash="dash"), name="x-linearization"))
        # y-linearization: vary y, hold x (line)
        fig.add_trace(go.Scatter3d(x=np.full_like(ys_x, x0), y=ys_x, z=z_lin_y, mode="lines", line=dict(color="#ff7f0e", width=4, dash="dash"), name="y-linearization"))
    # Layout
    scene = dict(xaxis_title="x", yaxis_title="y", zaxis_title="z", aspectmode="data",
                 xaxis=dict(showspikes=False), yaxis=dict(showspikes=False), zaxis=dict(showspikes=False))
    fig.update_layout(scene=dict(**scene, camera=dict(eye=dict(x=1.5, y=1.35, z=0.95), projection=dict(type="orthographic"))), margin=dict(l=0, r=0, t=28, b=80), legend=dict(orientation="h", y=-0.18, yanchor="top", x=0.5, xanchor="center"), title="Surface cross-section and linearization (3D)", width=650, height=480, uirevision="v1-3d")
    return fig
def _build_2d_v1():
    x0 = float(x0_v1.value)
    y0 = float(y0_v1.value)
    xs, ys, zs, z_lin, tvar, t0, z0, label = _slice_values_and_tangent(x0, y0, slice_axis_v1.value)
    fig2 = go.Figure()
    # Actual slice z(t)
    fig2.add_trace(go.Scatter(x=tvar, y=zs, mode="lines", line=dict(color="#FF4136", width=4), name="z(t) on slice"))
    # Tangent line at t0
    if show_linear_v1.value:
        fig2.add_trace(go.Scatter(x=tvar, y=z_lin, mode="lines", line=dict(color="#1f77b4", width=3, dash="dash"), name="linearization at t0"))
    # Highlight point
    fig2.add_trace(go.Scatter(x=[t0], y=[z0], mode="markers", marker=dict(size=8, color="#111111"), name="(t0, z0)"))
    fig2.update_layout(xaxis_title="t (selected variable)", yaxis_title="z", title=f"{label} — side view", width=340, height=260, margin=dict(l=40, r=10, t=40, b=40), showlegend=False, uirevision="v1-2d")
    return fig2

# Build a 2D slice explicitly for a chosen axis (so we can show both)
def _build_2d_v1_axis(axis: str):
    x0 = float(x0_v1.value)
    y0 = float(y0_v1.value)
    xs, ys, zs, z_lin, tvar, t0, z0, label = _slice_values_and_tangent(x0, y0, axis)
    fig2 = go.Figure()
    slice_color = COLOR_X if axis == "y" else COLOR_Y
    fig2.add_trace(go.Scatter(x=tvar, y=zs, mode="lines", line=dict(color=slice_color, width=4), name="slice z(t)"))
    if show_linear_v1.value:
        lin_color = COLOR_X if axis == "y" else COLOR_Y
        fig2.add_trace(go.Scatter(x=tvar, y=z_lin, mode="lines", line=dict(color=lin_color, width=3, dash="dot"), name="linearization at t0"))
    fig2.add_trace(go.Scatter(x=[t0], y=[z0], mode="markers", marker=dict(size=8, color="#111111"), name="(t0, z0)"))
    if axis == "y":
        ttl = "Along x (hold y)"
    else:
        ttl = "Along y (hold x)"
    fig2.update_layout(xaxis_title="t", yaxis_title="z", title=ttl, width=300, height=220, margin=dict(l=32, r=8, t=28, b=28), showlegend=False, uirevision=f"v1-2d-{axis}")
    return fig2

_is_rendering_v1 = False
def _render_v1(*args):
    global _is_rendering_v1
    if _is_rendering_v1:
        return
    _is_rendering_v1 = True
    # Sync cached Z with chosen surface for this view
    try:
        if 'surface_dropdown' in globals() and surface_dropdown.value != surface_dropdown_v1.value:
            surface_dropdown.value = surface_dropdown_v1.value
    except Exception:
        pass
    _update_z_stats_for_current_surface()
    with out3d_v1:
        clear_output(wait=True)
        display(_build_3d_v1())
    with out2d_x_v1:
        clear_output(wait=True)
        _figx = _build_2d_v1_axis('y')  # along x (hold y)
        _figx.show(config=dict(displayModeBar=False, displaylogo=False))
    with out2d_y_v1:
        clear_output(wait=True)
        _figy = _build_2d_v1_axis('x')  # along y (hold x)
        _figy.show(config=dict(displayModeBar=False, displaylogo=False))
    _is_rendering_v1 = False
# Wire observers
surface_dropdown_v1.observe(_render_v1, names="value")
x0_v1.observe(_render_v1, names="value")
y0_v1.observe(_render_v1, names="value")
#slice_axis_v1.observe(_render_v1, names="value")
show_linear_v1.observe(_render_v1, names="value")
controls_v1 = widgets.VBox([widgets.HTML("<b>Visualization 1:</b> Surface cross-section and one-variable linearization."),widgets.HBox([surface_dropdown_v1]),widgets.HBox([x0_v1, y0_v1, show_linear_v1]),])
side_panels_v1 = widgets.VBox([widgets.HTML("<b>Side panels:</b>"),out2d_x_v1,out2d_y_v1,])
plots_row_v1 = widgets.HBox([out3d_v1,side_panels_v1,], layout=widgets.Layout(align_items='flex-start'))
layout_v1 = widgets.VBox([controls_v1,plots_row_v1,])

### Linearization and Partial Derivatives Visualization
Click buttons to change the axes, and move sliders for xs and ys to change the point which is being partially derivitized and linearized

In [10]:
_render_v1()

display(layout_v1)

### This is a 3D visualization of the level sets shown above

To understand what the tangent plane is, click the "Tangent Plane" Checkbox below. Then, change X and Y values in order to move your gradient into the desired location!

In [11]:
ui_levels = widgets.VBox([
    widgets.HTML("<b>3D Level Sets</b> — Clean (no tangent plane, gradients, or derivative lines)."),
    controls_row1_ls,
    plane_controls_ls,
    widgets.HBox([birds_eye_toggle_ls]),
    out3d_ls,
])

# Initial sync from main UI (if present), then render and display
try:
    _sync_ls_from_main()
except Exception:
    render_levels()

display(ui_levels)


In [12]:
# Layout
controls_grad = widgets.HBox([surface_dropdown_g, x0_input_g, y0_input_g, show_tangent_plane_chk_g, birds_eye_toggle_g])
ui_grad = widgets.VBox([
    widgets.HTML("<b>3D Gradient Field</b> — Surface dropdown, (x0,y0) controls, tangent-plane toggle, and Bird’s-eye."),
    controls_grad,
    out3d_grad,
])

# Initial sync and render
_sync_grad_controls_from_main()
render_grad_view()
display(ui_grad)


In [13]:
render_all()
display(ui)